In [4]:
import numpy as np 
import pandas as pd 
import netCDF4
import pickle 
import os
import pandas as pd
import h5py 
import numpy as np

def Reshape( X ):
    x =[ ]
    for i in range(X.shape[1]):
        x.append(X[: , i  , :])
    return np.array(x)

def rescale( data  ,Ref ) :
    means = np.mean(Ref , axis = 0 )
    std = np.std( Ref , axis =0)
    return (data -means ) / std

def Dict_to_hdf(Dict , of ):
    HDF = h5py.File(of, 'w')
    for k in Dict :    
        test = HDF.create_dataset( k , data = Dict[k] )
    HDF.close()   


def nc_to_pd(nc):
    try:
        Keys = list(nc.keys())
    except:
        Keys = list(nc.variables)
    Panda ={ }
    for k in Keys:
        Panda[k]= np.array(nc[k] )    
    return pd.DataFrame(Panda)

def DF_bites_to_str(DF):
        
    for k in DF :
        if type(DF[k][0])  == np.bytes_ :
            DF[k]  = np.array( DF[k]).astype(str)
    return DF

def Decode( List, Format ):
    a = [ ]
    for l in List:
        a.append(   l.decode(Format))
    return np.array(a)
def Sum(L1 , L2):
    L = []
    for l in range(len(L1)) :
        L.append(L1[l] + L2[l]) 
    return L

def Split(DF):
    df_train = DF[DF['YEAR']  <= 2020  ]
    df_test = DF[DF['YEAR']  > 2020]

    df = {}

    df['train']  , df['test']  =  df_train , df_test


    X = {}

    X_train =[]
    X_test =[]

    X_DF = []

    for K in ord_Keys : 

        X['train'] = df['train'][ K  ].values
        X['test'] = df['test'][ K  ].values
        X_train.append(rescale(  X['train'] , X['train']  )) 
        X_test.append( rescale(  X['test'] , X['train']  ) )

        X_DF.append(rescale(   DF[ K  ].values , X['train']  ))

    obs_train  = np.array(df['train'][obs_key])
    obs_test = np.array(df['test'][obs_key])

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    X_DF  = np.array(X_DF)

    return  X_train , X_test , X_DF  , obs_train , obs_test


def Split_for_image(DF , df_image ):
    df_train = DF[DF['YEAR']  <= 2020  ]
    df_test = df_image 

    df = {}

    df['train']  , df['test']  =  df_train , df_test


    X = {}

    X_train =[]
    X_test =[]

    X_DF = []

    for K in ord_Keys : 

        X['train'] = df['train'][ K  ].values
        X['test'] = df['test'][ K  ].values
        X_train.append(rescale(  X['train'] , X['train']  )) 
        X_test.append( rescale(  X['test'] , X['train']  ) )

        X_DF.append(rescale(   DF[ K  ].values , X['train']  ))

    X_train = np.array(X_train).astype(float)
    X_image = np.array(X_test).astype(float)
    X_DF  = np.array(X_DF).astype(float)

    return  X_train , X_image    

def Read(DF ,  hdfs , ord_Keys ) : 
    
    for Keys in ord_Keys:
        for k in Keys:
            for hdf in hdfs :
                if k in list(hdf.keys()):
                    DF[k] = np.array(hdf[k])
    return DF




In [111]:

class Dataframe_multi_dim :

    def __init__(self  , Dict ):
        self.df  = Dict
        self.Keys = list(self.df.keys())
        
    def Querry(  self , Key  ,  condition  , value) : 
        
        self.df_k = pd.DataFrame(self.df[Key] , columns= [Key])

        if condition  == '>=' : 
            querry = self.df_k[self.df_k[Key]  >= value ] 
        if condition  == '>' : 
            querry = self.df_k[self.df_k[Key]  > value ] 
        if condition  == '==' : 
            querry = self.df_k[self.df_k[Key]  == value ] 
        if condition  == '<' : 
            querry = self.df_k[self.df_k[Key]  < value ] 
        if condition  == '<=' :
            querry = self.df_k[self.df_k[Key]  <= value ] 
        if condition  == '!=' :
            querry = self.df_k[self.df_k[Key]  != value ] 
        self.Index = np.array(querry.index)
        self.out_df  ={ }
        
        for k in self.Keys  :
            self.out_df[k]  = np.array(self.df[k])[self.Index]
        self.df  = self.out_df
        self.Keys = list(self.df.keys())
        return Dataframe_multi_dim(self.df )




In [6]:

Folder = "D:/Projects/precipitation_AUS/dataset/0025/"
files = os.listdir(Folder)
hdfs = []

for file in files :
    hdf = h5py.File(Folder + file , 'r+')
    hdfs.append( hdf)
    print(hdf.keys())
    

<KeysViewHDF5 ['CAL_P', 'CAL_P(-1)', 'CAL_P(-2)', 'CAL_P(-3)', 'CAL_P(-4)', 'CAL_P(-5)', 'CAL_P_ave', 'DAY', 'DEWP', 'DEWP_ATTRIBUTES', 'Date', 'ELEVATION', 'ERA5_cape', 'ERA5_cape(-1)', 'ERA5_cape(-2)', 'ERA5_cape(-3)', 'ERA5_cape(-4)', 'ERA5_cape(-5)', 'ERA5_msl', 'ERA5_msl(-1)', 'ERA5_msl(-2)', 'ERA5_msl(-3)', 'ERA5_msl(-4)', 'ERA5_msl(-5)', 'ERA5_t2m', 'ERA5_t2m(-1)', 'ERA5_t2m(-2)', 'ERA5_t2m(-3)', 'ERA5_t2m(-4)', 'ERA5_t2m(-5)', 'ERA5_tcw', 'ERA5_tcw(-1)', 'ERA5_tcw(-2)', 'ERA5_tcw(-3)', 'ERA5_tcw(-4)', 'ERA5_tcw(-5)', 'ERA5_tp', 'ERA5_tp(-1)', 'ERA5_tp(-2)', 'ERA5_tp(-3)', 'ERA5_tp(-4)', 'ERA5_tp(-5)', 'FRSHTT', 'GUST', 'HQ_P', 'HQ_P(-1)', 'HQ_P(-2)', 'HQ_P(-3)', 'HQ_P(-4)', 'HQ_P(-5)', 'LATITUDE', 'LONGITUDE', 'LSTM_RA_0', 'LSTM_RA_0(-1)', 'LSTM_RA_0(-2)', 'LSTM_RA_0(-3)', 'LSTM_RA_0(-4)', 'LSTM_RA_1', 'LSTM_RA_2', 'LSTM_RA_3', 'LSTM_RA_3(-1)', 'LSTM_RA_3(-2)', 'LSTM_RA_3(-3)', 'LSTM_RA_3(-4)', 'LSTM_RA_4', 'LSTM_RA_4(-4)', 'MAX', 'MAX_ATTRIBUTES', 'MIN', 'MIN_ATTRIBUTES', 'MON

In [7]:

Folder = "D:/Projects/precipitation_AUS/dataset/0025/"
files = os.listdir(Folder)
hdfs = []

for file in files :
    hdf = h5py.File(Folder + file , 'r+')
    hdfs.append( hdf)
    print(hdf.keys())
    
needed_keys = [ 'NAME'  , 'LATITUDE' , 'LONGITUDE' ,  'YEAR'  , 'MONTH'   ,  'DAY'  , 'PRCP'  , 'STATE_CODE' , 'PDIR'  , 'ERA5_tp'  , 
'ERA5_cape' , 'ERA5_tcw'   , 'CAL_P' ] 

DF = pd.DataFrame()
for k in needed_keys :
    for hdf in hdfs :
        if (k not in list(DF.keys())) and (k in list(hdf.keys())):
            DF[k] = np.array(hdf[k])

In [114]:
MDF =  Dataframe_multi_dim(DF)
new =  MDF.Querry('YEAR'  ,  '>='  , 2021 )

In [117]:
pd.DataFrame = new.df

In [105]:
MDF.df

{'NAME': array([b'ADELAIDE INTERNATIONAL', b'ADELAIDE INTERNATIONAL',
        b'ADELAIDE INTERNATIONAL', ..., b'DALWALLINU COMPARISON',
        b'DALWALLINU COMPARISON', b'DALWALLINU COMPARISON'], dtype=object),
 'LATITUDE': array([-34.945    , -34.945    , -34.945    , ..., -30.2833333,
        -30.2833333, -30.2833333]),
 'LONGITUDE': array([138.530556 , 138.530556 , 138.530556 , ..., 116.6666666,
        116.6666666, 116.6666666]),
 'YEAR': array([2021, 2021, 2021, ..., 2024, 2024, 2024]),
 'MONTH': array([1, 1, 1, ..., 1, 1, 1]),
 'DAY': array([ 1,  2,  3, ..., 29, 30, 31]),
 'PRCP': array([0., 0., 0., ..., 0., 0., 0.]),
 'STATE_CODE': array([1, 1, 1, ..., 5, 5, 5], dtype=int64),
 'PDIR': array([2., 0., 1., ..., 0., 0., 0.]),
 'ERA5_tp': array([1.20376528e-07, 1.12852995e-05, 2.40753056e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 'ERA5_cape': array([0.20641509, 1.95449287, 1.83193391, ..., 0.        , 0.        ,
        0.        ]),
 'ERA5_tcw': array([18

In [97]:
mdf_2 =  Dataframe_multi_dim(new)

In [98]:
mdf_2.Querry('MONTH'  ,  '=='  , 1 ) 

{'NAME': array([b'ADELAIDE INTERNATIONAL', b'ADELAIDE INTERNATIONAL',
        b'ADELAIDE INTERNATIONAL', ..., b'DALWALLINU COMPARISON',
        b'DALWALLINU COMPARISON', b'DALWALLINU COMPARISON'], dtype=object),
 'LATITUDE': array([-34.945    , -34.945    , -34.945    , ..., -30.2833333,
        -30.2833333, -30.2833333]),
 'LONGITUDE': array([138.530556 , 138.530556 , 138.530556 , ..., 116.6666666,
        116.6666666, 116.6666666]),
 'YEAR': array([2021, 2021, 2021, ..., 2024, 2024, 2024]),
 'MONTH': array([1, 1, 1, ..., 1, 1, 1]),
 'DAY': array([ 1,  2,  3, ..., 29, 30, 31]),
 'PRCP': array([0., 0., 0., ..., 0., 0., 0.]),
 'STATE_CODE': array([1, 1, 1, ..., 5, 5, 5], dtype=int64),
 'PDIR': array([2., 0., 1., ..., 0., 0., 0.]),
 'ERA5_tp': array([1.20376528e-07, 1.12852995e-05, 2.40753056e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 'ERA5_cape': array([0.20641509, 1.95449287, 1.83193391, ..., 0.        , 0.        ,
        0.        ]),
 'ERA5_tcw': array([18

In [82]:
mdf_2.df_k

,0
0,1
1,1
2,1
3,1
4,1
...,...
498184,1
498185,1
498186,1
498187,1


In [60]:
MDF.df['YEAR']

array([2021, 2021, 2021, ..., 2024, 2024, 2024])

In [50]:
qmdf.df

{'NAME': array([b'ADELAIDE INTERNATIONAL', b'ADELAIDE INTERNATIONAL',
        b'ADELAIDE INTERNATIONAL', ..., b'DALWALLINU COMPARISON',
        b'DALWALLINU COMPARISON', b'DALWALLINU COMPARISON'], dtype=object),
 'LATITUDE': array([-34.945    , -34.945    , -34.945    , ..., -30.2833333,
        -30.2833333, -30.2833333]),
 'LONGITUDE': array([138.530556 , 138.530556 , 138.530556 , ..., 116.6666666,
        116.6666666, 116.6666666]),
 'YEAR': array([2021, 2021, 2021, ..., 2024, 2024, 2024]),
 'MONTH': array([1, 1, 1, ..., 1, 1, 1]),
 'DAY': array([ 1,  2,  3, ..., 29, 30, 31]),
 'PRCP': array([0., 0., 0., ..., 0., 0., 0.]),
 'STATE_CODE': array([1, 1, 1, ..., 5, 5, 5], dtype=int64),
 'PDIR': array([2., 0., 1., ..., 0., 0., 0.]),
 'ERA5_tp': array([1.20376528e-07, 1.12852995e-05, 2.40753056e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 'ERA5_cape': array([0.20641509, 1.95449287, 1.83193391, ..., 0.        , 0.        ,
        0.        ]),
 'ERA5_tcw': array([18

In [18]:
pd.DataFrame(qmdf)

,NAME,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,PRCP,STATE_CODE,PDIR,ERA5_tp,ERA5_cape,ERA5_tcw,CAL_P
0,b'ADELAIDE INTERNATIONAL',-34.945000,138.530556,2021,1,1,0.0,1,2.0,1.203765e-07,0.206415,18.760378,0.212751
1,b'ADELAIDE INTERNATIONAL',-34.945000,138.530556,2021,1,2,0.0,1,0.0,1.128530e-05,1.954493,17.083668,0.000000
2,b'ADELAIDE INTERNATIONAL',-34.945000,138.530556,2021,1,3,0.0,1,1.0,2.407531e-07,1.831934,19.017394,0.149505
3,b'ADELAIDE INTERNATIONAL',-34.945000,138.530556,2021,1,4,0.0,1,0.0,4.784967e-06,1.077229,19.719130,0.000000
4,b'ADELAIDE INTERNATIONAL',-34.945000,138.530556,2021,1,5,0.0,1,0.0,2.106589e-07,0.245118,14.335306,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498184,b'DALWALLINU COMPARISON',-30.283333,116.666667,2024,1,27,0.0,5,0.0,0.000000e+00,0.000000,13.975645,0.000000
498185,b'DALWALLINU COMPARISON',-30.283333,116.666667,2024,1,28,0.0,5,0.0,0.000000e+00,0.000000,17.875948,0.000000
498186,b'DALWALLINU COMPARISON',-30.283333,116.666667,2024,1,29,0.0,5,0.0,0.000000e+00,0.000000,17.441494,0.000000
498187,b'DALWALLINU COMPARISON',-30.283333,116.666667,2024,1,30,0.0,5,0.0,0.000000e+00,0.000000,21.580912,0.000000


In [11]:
k = 'ERA5_tp'
np.mean(DF[k])

8.707034736754034e-05

In [12]:
# hdfs for images
Folder = "D:/Projects/precipitation_AUS/dataset/23feb2022/"
files_images = os.listdir(Folder)
hdfs_image = []

for file in files_images :
    hdf = h5py.File(Folder + file , 'r+')
    hdfs_image.append( hdf)
    print(hdf.keys())


<KeysViewHDF5 ['DAY', 'LATITUDE', 'LONGITUDE', 'MONTH', 'YEAR']>
<KeysViewHDF5 ['ACO-MLP']>
<KeysViewHDF5 ['CNN3', 'CNN3C', 'CNN5', 'CNN5C', 'CNN7', 'CNN7C']>
<KeysViewHDF5 ['ERA5_cape', 'ERA5_cape_Ci_box(3, 3)', 'ERA5_cape_Ci_box(5, 5)', 'ERA5_cape_Ci_box(7, 7)', 'ERA5_cape_Cj_box(3, 3)', 'ERA5_cape_Cj_box(5, 5)', 'ERA5_cape_Cj_box(7, 7)', 'ERA5_cape_ave_box(3, 3)', 'ERA5_cape_ave_box(5, 5)', 'ERA5_cape_ave_box(7, 7)', 'ERA5_cape_std_box(3, 3)', 'ERA5_cape_std_box(5, 5)', 'ERA5_cape_std_box(7, 7)', 'ERA5_t2m', 'ERA5_t2m_Ci_box(3, 3)', 'ERA5_t2m_Ci_box(5, 5)', 'ERA5_t2m_Ci_box(7, 7)', 'ERA5_t2m_Cj_box(3, 3)', 'ERA5_t2m_Cj_box(5, 5)', 'ERA5_t2m_Cj_box(7, 7)', 'ERA5_t2m_ave_box(3, 3)', 'ERA5_t2m_ave_box(5, 5)', 'ERA5_t2m_ave_box(7, 7)', 'ERA5_t2m_std_box(3, 3)', 'ERA5_t2m_std_box(5, 5)', 'ERA5_t2m_std_box(7, 7)', 'ERA5_tcw', 'ERA5_tcw_Ci_box(3, 3)', 'ERA5_tcw_Ci_box(5, 5)', 'ERA5_tcw_Ci_box(7, 7)', 'ERA5_tcw_Cj_box(3, 3)', 'ERA5_tcw_Cj_box(5, 5)', 'ERA5_tcw_Cj_box(7, 7)', 'ERA5_tcw_ave_b

In [ ]:
# load a model
cnn_names = ['CNN3'  , 'CNN5'  , 'CNN7'  , 'CNN3C'  , 'CNN5C' ,  'CNN7C' ]
out_dict = {}

import tensorflow as tf
import keras

for name in cnn_names : 
    model = tf.keras.models.load_model( 'D:/Projects/precipitation_AUS/models-2/'+name+ r'.keras' )
    model_meta = h5py.File(r"D:\Projects\precipitation_AUS\models-2/"+name+r".hdf5" )

    #model.summary()    

    model_name = str(np.array(model_meta['name']))
    print(model_name)
    ord_Keys = []
    binary_keys = np.array(model_meta['ord_keys' ])
    for k in binary_keys  : ord_Keys.append( Decode(k  , 'utf-8') ) 

    DF =  Read( DF , hdfs , ord_Keys )
    df_image = Read( pd.DataFrame() , hdfs_image , ord_Keys )
    X_train , X_image = Split_for_image(DF , df_image )
    x_train =  Reshape(X_train)
    x_image  = Reshape(X_image)
    
    out_dict[model_name] = model.predict( x_image).T[0]



# Run models for DataFrame

In [14]:
cnn_names = ['CNN3'  , 'CNN5'  , 'CNN7'  , 'CNN3C'  , 'CNN5C' ,  'CNN7C' ]
out_dict = {}

import tensorflow as tf
import keras
obs_key = 'PRCP'
for name in cnn_names : 
    model = tf.keras.models.load_model( 'D:/Projects/precipitation_AUS/models-2/'+name+ r'.keras' )
    model_meta = h5py.File(r"D:\Projects\precipitation_AUS\models-2/"+name+r".hdf5" )

    #model.summary()    

    model_name = str(np.array(model_meta['name']))
    print(model_name[2:-1])
    
    ord_Keys = []
    binary_keys = np.array(model_meta['ord_keys' ])
    for k in binary_keys  : ord_Keys.append( Decode(k  , 'utf-8') ) 

    DF =  Read( DF , hdfs , ord_Keys )
    X_train , X_test , X_DF  , obs_train , obs_test = Split(DF)
    x = Reshape(X_DF)
    out_dict[model_name[2:-1]] = model.predict(x)
    

CNN3
47252/47252 ━━━━━━━━━━━━━━━━━━━━ 40s 845us/step
CNN5
47252/47252 ━━━━━━━━━━━━━━━━━━━━ 50s 1ms/step
CNN7


C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF[k] = np.array(hdf[k])
C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF[k] = np.array(hdf[k])
C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at 

47252/47252 ━━━━━━━━━━━━━━━━━━━━ 93s 2ms/step
CNN3C


C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF[k] = np.array(hdf[k])
C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF[k] = np.array(hdf[k])
C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at 

47252/47252 ━━━━━━━━━━━━━━━━━━━━ 66s 1ms/step
CNN5C


C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF[k] = np.array(hdf[k])
C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF[k] = np.array(hdf[k])
C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at 

47252/47252 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step
CNN7C


C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF[k] = np.array(hdf[k])
C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF[k] = np.array(hdf[k])
C:\Users\S4055367\AppData\Local\Temp\ipykernel_14928\781912636.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at 

47252/47252 ━━━━━━━━━━━━━━━━━━━━ 114s 2ms/step


array([[[ 4.43639234e+00, -5.34347749e-04,  2.74316940e-01,
         -2.79031000e-01,  2.08655120e-01,  1.93188567e+00],
        [ 3.63487988e+00, -2.87986339e-02,  3.57283312e-01,
         -2.79364976e-01,  2.25047113e-01,  1.94004561e+00],
        [ 2.22262989e+00,  5.67718156e-01,  1.37948206e+00,
         -2.14368613e-01,  1.11486940e+00,  1.29068752e+00]],

       [[-2.54941745e-01, -1.25016500e-01,  1.52087096e-01,
         -3.18820473e-01, -3.71991168e-01,  1.39133058e+00],
        [-2.52625463e-01, -1.54701688e-01,  2.31698007e-01,
         -3.21704654e-01, -3.71306233e-01,  1.39537505e+00],
        [-2.20789844e-01,  1.32682033e-02,  5.04526628e-01,
         -3.18023458e-01,  1.46995911e+00,  3.60582077e-01]],

       [[-2.70338066e-01, -2.49498653e-01, -2.41480738e-01,
         -2.86600777e-01, -3.93626829e-01, -7.24479024e-01],
        [-2.76831614e-01, -2.52626285e-01, -2.52844839e-01,
         -2.93414621e-01, -3.71354781e-01, -7.17242794e-01],
        [-3.09701535e-01, -3

In [14]:
model_name[2:-1]

'NN7C'

In [15]:

def Dict_to_hdf(Dict , of ):
    HDF = h5py.File(of, 'w')
    for k in Dict :    
        test = HDF.create_dataset( k , data = Dict[k] )
        print( k)
    HDF.close()   

In [16]:
Dict_to_hdf(out_dict , r"D:\Projects\precipitation_AUS\dataset\0025\joint-all-CNNs.hdf5" ) 

CNN3
CNN5
CNN7
CNN3C
CNN5C
CNN7C


# ACOR-MLP

In [17]:
class MLP :
    def __init__(self, nn_shape , inp_feat ):
        self.nn_shape = nn_shape

        self.inp_feat = inp_feat

        self.W_Shapes = []
        self.B_Shapes = []

        for i in range(len(self.nn_shape)):
            if i == 0 :
                self.W_Shapes .append(  (self.inp_feat,  self.nn_shape[i])    )    
            else:
                self.W_Shapes .append(  (self.nn_shape[i-1],self.nn_shape[i])    )
            self.B_Shapes .append(   self.nn_shape[i] )
        self.W_Shapes.append((self.nn_shape[-1] , 1))
        self.B_Shapes .append(  1 )
        self.w_params =[]

        for I in self.W_Shapes:
            self.w_params.append(  I[0]*I[1]    )

        self.Sum_params = sum(self.w_params ) +sum(self.B_Shapes)
        
    def update(self, solution):
        ii = 0
        self.W = []
        for i in range(len(self.W_Shapes)) : 
            a = np.array(solution[ ii  : ii  + self.w_params[i] ])
            self.W.append( np.reshape(a, self.W_Shapes[i]))
            ii = ii  + self.w_params[i] 
        self.B= []
        for i in range(len(self.B_Shapes)) :
            a = np.array(solution[ ii  : ii  + self.B_Shapes[i] ])
            self.B.append( a )
            ii = ii  +  self.B_Shapes[i]
        self.number_of_layers = len(self.W)
        
    def feed_forward(self , X ):
        for i in range(self.number_of_layers):
            h = numpy.dot(X,numpy.array(self.W[i])) + self.B[i]
            if i!= self.number_of_layers-1:
                # Xa = numpy.tanh(h)
                X = numpy.maximum(h,0) 
        return X
    

In [19]:
CNN7C =  np.array(hdfs_image[1][ 'CNN7C'] )

NameError: name 'hdfs_image' is not defined

In [21]:
CNN7C =  np.array(out_dict[ 'CNN7C'] )

In [22]:
import pickle
with open(f'../models-2/ACO-MLP.pickle', 'rb') as file2:
    model_aco_nn = pickle.load(file2)

In [23]:
import numpy
model_aco_nn.feed_forward( np.array( [CNN7C]  ).T )

array([[[5.23006747],
        [5.97031466],
        [0.16191503],
        ...,
        [0.15570654],
        [0.16803729],
        [0.14672869]]])

In [24]:
out_dict ={}
out_dict['ACO-MLP'] = model_aco_nn.feed_forward( np.array( [CNN7C]  ).T ).T[0]

In [25]:
Dict_to_hdf(out_dict , r"D:\Projects\precipitation_AUS\dataset\0025\joint-ACO.hdf5" ) 

ACO-MLP


In [26]:
out_dict

{'ACO-MLP': array([[5.23006747],
        [5.97031466],
        [0.16191503],
        ...,
        [0.15570654],
        [0.16803729],
        [0.14672869]])}

# DF to images 

In [2]:
import os 
import h5py
import tifffile as tff 
import numpy as np
import pandas as pd
# hdfs for images
Folder = "D:/Projects/precipitation_AUS/dataset/23feb2022/"
files_images = os.listdir(Folder)
hdfs_image = []

for file in files_images :
    hdf = h5py.File(Folder + file , 'r+')
    print(hdf.keys())
    hdfs_image.append( hdf)


<KeysViewHDF5 ['DAY', 'LATITUDE', 'LONGITUDE', 'MONTH', 'YEAR']>
<KeysViewHDF5 ['ACO-MLP']>
<KeysViewHDF5 ['CNN3', 'CNN3C', 'CNN5', 'CNN5C', 'CNN7', 'CNN7C']>
<KeysViewHDF5 ['ERA5_cape', 'ERA5_cape_Ci_box(3, 3)', 'ERA5_cape_Ci_box(5, 5)', 'ERA5_cape_Ci_box(7, 7)', 'ERA5_cape_Cj_box(3, 3)', 'ERA5_cape_Cj_box(5, 5)', 'ERA5_cape_Cj_box(7, 7)', 'ERA5_cape_ave_box(3, 3)', 'ERA5_cape_ave_box(5, 5)', 'ERA5_cape_ave_box(7, 7)', 'ERA5_cape_std_box(3, 3)', 'ERA5_cape_std_box(5, 5)', 'ERA5_cape_std_box(7, 7)', 'ERA5_t2m', 'ERA5_t2m_Ci_box(3, 3)', 'ERA5_t2m_Ci_box(5, 5)', 'ERA5_t2m_Ci_box(7, 7)', 'ERA5_t2m_Cj_box(3, 3)', 'ERA5_t2m_Cj_box(5, 5)', 'ERA5_t2m_Cj_box(7, 7)', 'ERA5_t2m_ave_box(3, 3)', 'ERA5_t2m_ave_box(5, 5)', 'ERA5_t2m_ave_box(7, 7)', 'ERA5_t2m_std_box(3, 3)', 'ERA5_t2m_std_box(5, 5)', 'ERA5_t2m_std_box(7, 7)', 'ERA5_tcw', 'ERA5_tcw_Ci_box(3, 3)', 'ERA5_tcw_Ci_box(5, 5)', 'ERA5_tcw_Ci_box(7, 7)', 'ERA5_tcw_Cj_box(3, 3)', 'ERA5_tcw_Cj_box(5, 5)', 'ERA5_tcw_Cj_box(7, 7)', 'ERA5_tcw_ave_b

In [3]:
names =  [  'CAL_P' ]

import tifffile as tff
df_image  = pd.DataFrame() 
for k in names:
    for hdf in hdfs_image :
        if k in list(hdf.keys()):
            df_image[k] = np.array(hdf[k])


In [4]:

new_shape = (313, 211)
folder  = r'D:\Projects\precipitation_AUS\IMAGES\run-2/'
for name in  names :
    image = np.reshape( df_image[name].values , new_shape)
    image  = np.maximum( image , 0  )
    if name == 'ERA5_tp'   : image = image *1000 *24 

    print(image.shape)
    of  = folder + name + '.tif'
    tff.imwrite(of , image )

(313, 211)


In [ ]:
np.absolute

In [5]:
df_image.keys()

RangeIndex(start=0, stop=0, step=1)